### Group Members
Xiaohan Wang (xw41)

Junjie  Yue  (junjiey4)

Xinyu Liu  (xinyul10)

ChenMing Xu  (cx13)

Import all the libraries and packages needed. 

In [1]:
%matplotlib inline  
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
import matplotlib.cm as cmx
import matplotlib.colors as colors

In [2]:
ufos = pd.read_csv('/Users/W/Downloads/ufo-scrubbed-geocoded-time-standardized-00.csv',
                  names=['date','city','state','country','shape','duration_seconds','duration','comment',
                       'report_date','latitude','longitude'],
                  parse_dates=['date','report_date'])

In [3]:
# process data tot turn it into strings from NAN's that are numbers
ufos = ufos.fillna({'country':'unknown','state':'unknown','shape':'unknwon'})
ufos

,date,city,state,country,shape,duration_seconds,duration,comment,report_date,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,unknown,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),unknown,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,2013-09-09 21:15:00,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,2013-09-30,36.165833,-86.784444
80328,2013-09-09 22:00:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,2013-09-30,43.613611,-116.202500
80329,2013-09-09 22:00:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,2013-09-30,38.297222,-122.284444
80330,2013-09-09 22:20:00,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,2013-09-30,38.901111,-77.265556


In [4]:
# Downsample the dataset to 1000 randomly selected rows for a faster execution
nsamples = 1000
downSampleMask = np.random.choice(range(len(ufos)-1), nsamples, replace=False)
downSampleMask
ufosDS = ufos.loc[downSampleMask]

Check the data types of different columns so that we can determine which variable is numerical and which one is categorical.

In [5]:
ufosDS.dtypes

date                datetime64[ns]
city                        object
state                       object
country                     object
shape                       object
duration_seconds           float64
duration                    object
comment                     object
report_date         datetime64[ns]
latitude                   float64
longitude                  float64
dtype: object

## Q1.

The following figures would allow users to choose which columns to plot the scatter plot accordingly. However, users would not choose whichever they like to make the plot, because we add some restrictions. 
* Firstly, we divide all the columns into two groups: categorical and numerical. 
* Then only allow numerical variables to be placed on the y-axis, and all variables could be placed on the x-axis. 
* To make the figure contain more info, we scale the values of some columns to colors accordingly, which means the color is the third dimension. 
    * In the first interactive figure, we only present choices of numerical values:'duration_seconds', 'latitude', 'longitude'.
    * In the second interactive figure, we add ccategorical variables to be scaled for coloring the scattered plot.

In [6]:
numerical_list = ['duration_seconds','latitude','longitude']  # pick up three columns that have numerical data

@ipywidgets.interact(col_name1=ufosDS.columns, col_name2=numerical_list, colormap = plt.colormaps(), 
                     col_name3=numerical_list )
# add three interactive choice bars to be x-values, y-values, and color styles
def scatter_plot(col_name1, col_name2, colormap, col_name3):        
    fig, ax = plt.subplots(figsize=(10,6))
    ax.scatter(ufosDS[col_name1], ufosDS[col_name2], c=np.log10(ufosDS[col_name3]), cmap=colormap)
    ax.set_xlabel(col_name1)
    ax.set_ylabel(col_name2)
    ax.set_title('Scatter Plot between {} and {}'.format(col_name1,col_name2), fontsize=15)
    fig.autofmt_xdate(rotation=90)
    plt.show()

interactive(children=(Dropdown(description='col_name1', options=('date', 'city', 'state', 'country', 'shape', …

For this part, we revised the code to get color values generated by some categorical values as well. The crux here is converting catogorical values into some numerical values that can be use to generate different colors. 

In [7]:
color_column_list = ['duration_seconds','latitude','longitude','city','state','country','shape']
# get all unique values of a getegorical variable and,
# use indexes of diferent values as the base to get color values that can be used in color maps accordingly. 

@ipywidgets.interact(col_name1=ufosDS.columns, col_name2=numerical_list, colormap = plt.colormaps(), col_name3=color_column_list)
def scatter_plot(col_name1, col_name2, colormap, col_name3):        
    if col_name3 in numerical_list:
        fig, ax = plt.subplots(figsize=(10,6))
        ax.scatter(ufosDS[col_name1], ufosDS[col_name2], c=np.log10(ufosDS[col_name3]), cmap=colormap)
    else:
        fig, ax = plt.subplots(figsize=(10,6))
        x = ufosDS[col_name1]
        y = ufosDS[col_name2]
        uniq = list(ufosDS[col_name3].unique())
        
        cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=colormap)
        
        for i in range(len(uniq)):
            index = ufosDS[col_name3] == uniq[i]
            ax.scatter(x[index],y[index],color=scalarMap.to_rgba(i),label=uniq[i])
    ax.set_xlabel(col_name1)
    ax.set_ylabel(col_name2)
    ax.set_title('scatter plot of {} and {}'.format(col_name1,col_name2), fontsize=15)
    plt.show()

interactive(children=(Dropdown(description='col_name1', options=('date', 'city', 'state', 'country', 'shape', …

## Q2.
We built a interactice figure to display binned, aggregate values of a column, and allow users to change the field to bin and the number of bins if the field is numerical variable. After selecting one column data and the number of bins, the distribution would show up.

In [8]:
num_name_list = ['duration_seconds', 'latitude','longitude'] # create a list for numerical data columns
bins_slider = ipywidgets.IntSlider(min = 1, max = 20, step = 1) # create a slider to control the number of bins
@ipywidgets.interact(fields = ufosDS.columns, bins = bins_slider)

# The funtion we create will judge whether the chosen column is or not numerical and do different operations
def hist_plot(fields, bins):        
    if fields in num_name_list:  # 'bin' a numerical column
        plt.clf()
        fig, ax = plt.subplots(figsize=(10,6))
        ufosDS.hist(column = fields, ax = ax, bins = bins, xrot = 90)
        plt.show()
        
    else:  # 'bin' a categorical column
        plt.clf()
        fig, ax = plt.subplots(figsize=(10,6))
        ufosDS[fields].value_counts().plot(kind = 'bar', ax = ax, title = fields, rot = 90)
        plt.show()

interactive(children=(Dropdown(description='fields', options=('date', 'city', 'state', 'country', 'shape', 'du…